## SQL Case Study 'Country Club' - Tier 2
### Springboard DSCT Unit 8.3 SQL Case Study 
#### Pablo Ruiz Lopez

In [1]:
#Installing sqlalchemy
#!pip install sqlalchemy

In [2]:
#Imports packages
from sqlalchemy import create_engine
from sqlalchemy import inspect
import pandas as pd

In [3]:
#Create engine
engine = create_engine('sqlite:///sqlite_db_pythonsqlite.db')
#Create inspector
inspector = inspect(engine)

In [4]:
#Extract table names from DB using engine
table_names_e = engine.table_names()

C:\Users\pablo\AppData\Local\Temp/ipykernel_98776/718763829.py:2: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  table_names_e = engine.table_names()


In [5]:
#Check out table names using engine
print(table_names_e)

['Bookings', 'Facilities', 'Members']


In [6]:
#Extract table names from DB using inspector
table_names_i = inspector.get_table_names()
#Check out table names using inspector
print(table_names_i)

['Bookings', 'Facilities', 'Members']


In [7]:
#Extract column names from tables using inspector
for table_name in table_names_i:
    for column in inspector.get_columns(table_name):
        print("Column: %s" % column['name'])

Column: bookid
Column: facid
Column: memid
Column: starttime
Column: slots
Column: facid
Column: name
Column: membercost
Column: guestcost
Column: initialoutlay
Column: monthlymaintenance
Column: memid
Column: surname
Column: firstname
Column: address
Column: zipcode
Column: telephone
Column: recommendedby
Column: joindate


C:\Users\pablo\AppData\Local\Temp/ipykernel_98776/4074279356.py:3: SAWarning: Could not instantiate type <class 'sqlalchemy.sql.sqltypes.INTEGER'> with reflected arguments ['4']; using no arguments.
  for column in inspector.get_columns(table_name):
C:\Users\pablo\AppData\Local\Temp/ipykernel_98776/4074279356.py:3: SAWarning: Could not instantiate type <class 'sqlalchemy.sql.sqltypes.INTEGER'> with reflected arguments ['1']; using no arguments.
  for column in inspector.get_columns(table_name):
C:\Users\pablo\AppData\Local\Temp/ipykernel_98776/4074279356.py:3: SAWarning: Could not instantiate type <class 'sqlalchemy.sql.sqltypes.INTEGER'> with reflected arguments ['2']; using no arguments.
  for column in inspector.get_columns(table_name):
C:\Users\pablo\AppData\Local\Temp/ipykernel_98776/4074279356.py:3: SAWarning: Could not instantiate type <class 'sqlalchemy.sql.sqltypes.INTEGER'> with reflected arguments ['5']; using no arguments.
  for column in inspector.get_columns(table_name):


In [8]:
# Executing test query and store records in DataFrame: df
df = pd.read_sql_query('SELECT * FROM Members', engine)
# Print head of DataFrame
print(df.head())
print(type(df))

   memid   surname firstname                       address  zipcode  \
0      0     GUEST     GUEST                         GUEST        0   
1      1     Smith    Darren    8 Bloomsbury Close, Boston     4321   
2      2     Smith     Tracy  8 Bloomsbury Close, New York     4321   
3      3    Rownam       Tim        23 Highway Way, Boston    23423   
4      4  Joplette    Janice    20 Crossing Road, New York      234   

        telephone recommendedby             joindate  
0  (000) 000-0000                2012-07-01 00:00:00  
1    555-555-5555                2012-07-02 12:02:05  
2    555-555-5555                2012-07-02 12:08:23  
3  (844) 693-0723                2012-07-03 09:32:15  
4  (833) 942-4710             1  2012-07-03 10:25:05  
<class 'pandas.core.frame.DataFrame'>


#### QUESTIONS TIER 2:

##### Q10: Produce a list of facilities with a total revenue less than 1000. The output of facility name and total revenue, sorted by revenue. Remember that there's a different cost for guests and members! 

In [9]:
#Works both for MySQL and SQLite3
query = """SELECT name, SUM(Cost) AS Revenue 
           FROM (SELECT name, facid, memid, slots, 
                       CASE WHEN memid = 0 THEN (guestcost * slots)
                       ELSE (membercost * slots) END AS Cost
             FROM Bookings
             LEFT JOIN Facilities USING (facid)
             LEFT JOIN Members USING (memid)) AS sub
          GROUP BY name HAVING Revenue < 1000
          ORDER BY Revenue;"""

fac_rev = pd.read_sql_query(query, engine)
fac_rev

,name,Revenue
0,Table Tennis,180
1,Snooker Table,240
2,Pool Table,270


##### Q11: Produce a report of members and who recommended them in alphabetic surname,firstname order

In [10]:
#Works only for SQLite3
query = """SELECT m1.surname || ' ' || m1.firstname AS MemName,
                  m2.surname || ' ' || m2.firstname AS RecommendedBy
           FROM Members AS m1
           LEFT JOIN Members AS m2 ON m1.recommendedby = m2.memid
           WHERE m1.memid != 0
           ORDER BY MemName;"""

#Query for MySQL
"""
SELECT CONCAT (m1.surname, ' ', m1.firstname) AS MemName,
       CASE WHEN m2.memid = 0 THEN NULL 
       ELSE CONCAT (m2.surname, ' ', m2.firstname) END AS RecommendedBy
FROM Members AS m1
JOIN Members AS m2 ON m1.recommendedby = m2.memid
WHERE m1.memid != 0
ORDER BY MemName;
"""
mem_recom = pd.read_sql_query(query, engine)
mem_recom

,MemName,RecommendedBy
0,Bader Florence,Stibbons Ponder
1,Baker Anne,Stibbons Ponder
2,Baker Timothy,Farrell Jemima
3,Boothe Tim,Rownam Tim
4,Butters Gerald,Smith Darren
5,Coplin Joan,Baker Timothy
6,Crumpet Erica,Smith Tracy
7,Dare Nancy,Joplette Janice
8,Farrell David,None
9,Farrell Jemima,None


##### Q12: Find the facilities with their usage by member, but not guests

In [12]:
#Query for SQLite3
query = """SELECT surname || ' ' || firstname AS MemName,
                  name AS FacName,
                  ROUND(SUM(slots) / 2.0, 1) AS Hours
           FROM Bookings
           LEFT JOIN Facilities USING (facid)
           LEFT JOIN Members USING (memid)
           WHERE memid != 0
           GROUP BY MemName, name;"""

#Query for MySQL
"""SELECT CONCAT (surname, ' ', firstname) AS MemName,
          name AS FacName,
          ROUND(SUM(slots) / 2.0, 1) AS Hours
   FROM Bookings
   LEFT JOIN Facilities USING (facid)
   LEFT JOIN Members USING (memid)
   WHERE memid != 0
   gROUP BY MemName, name;"""

fac_us = pd.read_sql_query(query, engine)
fac_us

,MemName,FacName,Hours
0,Bader Florence,Badminton Court,13.5
1,Bader Florence,Massage Room 2,2.0
2,Bader Florence,Pool Table,11.5
3,Bader Florence,Snooker Table,33.0
4,Bader Florence,Squash Court,2.0
...,...,...,...
197,Tupperware Hyacinth,Squash Court,1.0
198,Worthington-Smyth Henry,Badminton Court,7.5
199,Worthington-Smyth Henry,Massage Room 1,1.0
200,Worthington-Smyth Henry,Pool Table,18.5


##### Q13: Find the facilities usage by month, but not guests

In [18]:
#Query for SQLite3
query = """SELECT strftime('%m', starttime) AS Month, 
                  name AS FacName, 
                  ROUND(SUM(slots) / 2.0, 1) AS Hours
           FROM Bookings
           LEFT JOIN Facilities USING (facid)
           LEFT JOIN Members USING (memid)
           WHERE memid != 0
           GROUP BY strftime('%m', starttime), name;"""

#Query for MySQL
"""SELECT MONTHNAME(starttime) AS Month, 
          name AS FacName, 
          ROUND(SUM(slots) / 2.0, 1) AS Hours
   FROM Bookings
   LEFT JOIN Facilities USING (facid)
   LEFT JOIN Members USING (memid)
   WHERE memid != 0
   GROUP BY MONTH(starttime), name;"""

fac_us_m = pd.read_sql_query(query, engine)
fac_us_m

,Month,FacName,Hours
0,07,Badminton Court,82.5
1,07,Massage Room 1,83.0
2,07,Massage Room 2,4.0
3,07,Pool Table,55.0
4,07,Snooker Table,70.0
5,07,Squash Court,25.0
6,07,Table Tennis,49.0
7,07,Tennis Court 1,100.5
8,07,Tennis Court 2,61.5
9,08,Badminton Court,207.0
